In [1]:
import sys
sys.path.insert(0,"/home/shackste/galaxy-generator/python_modules/")

import matplotlib.pyplot as plt

plt.style.use('dark_background')

from pdb import set_trace

In [2]:
from labeling import ConsiderGroups

cl = ConsiderGroups([1,2,3,4,5,6,7])
cl.get_considered_labels()


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [3]:
from train_generator import train_biggan

train_biggan()


!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under galaxy-generator/data/
the results will be placed there, too.

Adding attention layer in G at resolution 64
Param count for Gs initialized parameters: 33258627
Param count for Ds initialized parameters: 19560129


epoch 0:   0%|          | 0/55419 [00:00<?, ?it/s]/home/shackste/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

epochs:   0%|          | 0/2 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch
from tqdm import trange, tqdm

from sampler import generate_latent
from labeling import generate_labels
from big.BigGAN2 import Generator, Discriminator
from big.losses import generator_loss, discriminator_loss
from image_classifier import ImageClassifier

labels_dim = 37
latent_dim = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator(dim_z=latent_dim).to(device)
discriminator = Discriminator().to(device)
classifier = ImageClassifier().to(device)
classifier.load() ## use pretrained classifier

loss_class = torch.nn.MSELoss()



def train_step(images: torch.Tensor, labels: torch.Tensor):
    latent = generate_latent(labels.shape[0], latent_dim, sigma=False)
    labels_fake = generate_labels(labels.shape[0])
    label_transformed_fake_gen = generator.transform_labels(labels_fake)
    generated_images = generator(latent, label_transformed_fake_gen)

    #disc training
    label_transformed_fake_dis = discriminator.transform_labels(labels_fake)
    label_transformed_real_dis = discriminator.transform_labels(labels)

    prediction_fake = discriminator(generated_images.detach(), label_transformed_fake_dis).view(-1) 
    prediction_real = discriminator(images, label_transformed_real_dis).view(-1) 

    d_loss_real, d_loss_fake =  discriminator_loss(prediction_fake, prediction_real)
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    
    discriminator.optim.step()


    #gen training

    prediction = discriminator(generated_images, label_transformed_fake_dis.detach()).view(-1)
    labels_prediction = classifier.predict(generated_images)

    g_loss = generator_loss(prediction)
    g_loss += loss_class(labels_prediction, labels)
    
    g_loss.backward()

    generator.optim.step()
    
def train_epoch(data_loader):
    for images, labels in tqdm(data_loader_train, desc=f"epoch {epoch}"):
        train_step(images, labels)
